<a href="https://colab.research.google.com/github/YakshRathod/NULLCLASS-Internship/blob/Task-5/Task_5_Tokenizer.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from sentence_transformers import SentenceTransformer
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt

img_dir = "/content/drive/MyDrive/Nullclass internship/Task 1/Shapes Dataset"
img_files = sorted([f for f in os.listdir(img_dir) if f.lower().endswith('.png')])
img_paths = [os.path.join(img_dir, fname) for fname in img_files]

# Shape-only captions for each file
captions = [fname.split("_")[0] for fname in img_files]

# Double-check all lists are the same length (alignment!)
assert len(img_paths) == len(captions), "IMG/LABEL/EMB mismatch in lengths!"

# --- Embedding ---
model = SentenceTransformer('all-mpnet-base-v2')
embeddings = model.encode(captions, show_progress_bar=True)
embeddings = np.array(embeddings)

# --- Verify uniqueness ---
arrs = [tuple(e) for e in embeddings]
unique_count = len(set(arrs))
print(f"Total embeddings: {len(embeddings)} | Unique: {unique_count}")
if unique_count == len(embeddings):
    print("All embeddings are unique (no repeated vectors).")
else:
    print(f"WARNING: {len(embeddings)-unique_count} duplicate embeddings detected.")

# ---- Save aligned files ----
np.save("/content/drive/MyDrive/Nullclass internship/Task 5/shape_label_embeddings.npy", embeddings)
with open("/content/drive/MyDrive/Nullclass internship/Task 5/shape_labels.txt", "w") as f:
    for label in captions:
        f.write(label + "\n")
print("Embeddings and labels aligned, saved.")

# ---- Optional: Spot-check a few ----
import random
for i in random.sample(range(len(img_paths)), 4):
    label = captions[i]
    emb = embeddings[i]
    img = Image.open(img_paths[i]).convert('L').resize((128,128))
    plt.imshow(img, cmap='gray')
    plt.title(f"{label} (embedding[:3]={emb[:3]})")
    plt.axis('off')
    plt.show()
